## 개요 ##


- 빅데이터 수집 및 전처리.
- 머신러닝 앙상블 알고리즘 중 XGBoost 사용.
- 랜덤 포레스트와 XGBoost 알고리즘으로 아파트 매매 가격 예측 인공지능 모델을 제작 및 비교.


## 데이터 ##


- 데이터 세트를 제공하는 플랫폼인 캐글(kaggle)을 통해  오픈 API를 이용하여 데이터 불러오기


- 캐글 홈페이지(www.kaggle.com)에 접속 -> 로그인 -> 오른쪽 상단의 프로필 ‘Your Profile’
- Your Profile에 들어가 ‘Edit Public Profile’ 클릭 -> settings -> API 탭의 ‘Create New API Token’ 을 클릭  
-‘kaggle.json’을 다운로드


## 앙상블 머신러닝 랜덤 포레스트 ##


- 랜덤 포레스트는 의사결정 트리 알고리즘 여러 개가 결합된 앙상블 학습 방법으로, 분류에 사용된다.
- 랜덤 포레스트는 학습 과정에서 구성한 다수의 결정 트리로부터 평균 예측 값을 출력함으로써 회귀 분석, 즉, 수치를 예측하는 것도 가능하다.
- 가장 큰 특징은 무작위성(Random)에 의해 결정 트리들이 서로 다른 특성을 갖는다는 점이다.
- 이 특성으로 각 트리들의 예측이 비상관화되며, 결과적으로 일반화 성능을 향상시킨다.


## 앙상블 머신러닝 에이다 부스트(agaboost) ##


- XGBoost를 이해해기 위해 에이다 부스트를 먼저 알아야 한다.
- 부스팅은 약한 분류기를 세트로 붂어 정확도를 예측하는 기법이다.
- 약한 분류기들은 한 번에 하나씩 순차적으로 학습을 진행한다.
- 약한 분류기는 노드 하나에 두 개의 리프를 지닌 단순 구조의 트리로 '스텀프'라고 부른다.
- 먼저 학습된 분류기는 제대로 분류를 해 내는 데이터와 제대로 분류를 해 내지 못하는 데이터들이 발생한다.
- 이들을 다음 분류기에 전달한다.
- 다음 분류기는 이전 분류기로부터 받은 정보를 활용해 잘 분류해 내지 못한 데이터들을 견산하는 데 가중치를 높인다.
- 최종 분류기는 이전에 학습한 약한 분류기들에 각각의 가중치를 적용하고 조합하여 학습을 진행한다. 이를 에이다 부스트라 한다.
- 결과적으로 성능이 낮은 약한 분류기들을 조합하여 최종적으로 조금 더 성능이 좋은 강한 분류기 하나를 만드는 것이다.


## 앙상블 머신러닝 XGBoost ##


- XGBoost는 트리 기반의 앙상블 머신러닝 알고리즘이다.
- 기존 글라디언트 부스팅 알고리즘에 CART(Classification and Regression Trees) 모형을 기반으로 병렬 처리가 가능하게 만들어져 학습 시간을 획기적으로 줄인다.
- XGBoost의 약한 분류기는 데이터의 하위 집합에서 학습되고 예측을 수행하는 간단한 의사결정 트리 모델이다.
- 이러한 약한 분류기의 앙상블을 사용하여 최종 예측을 수행하고, 여러 모델의 강점을 결합하여 보다 정확한 모델을 생성한다.

In [53]:
import os

# 현재 디렉토리에서 'kaggle.json' 위치 지정
os.environ["KAGGLE_CONFIG_DIR"] = os.getcwd()

print("Kaggle 환경변수가 설정되었습니다!")


Kaggle 환경변수가 설정되었습니다!


In [54]:


#  재생성 방지 

In [55]:
# import zipfile

# with zipfile.ZipFile("korean-real-estate-transaction-data.zip", "r") as zip_ref:
#     zip_ref.extractall(".")  # 현재 디렉토리에 압축 해제
# print("압축 해제 완료!")



In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("./Apart Deal.csv")


df.head()

C:\Users\3호실-09\AppData\Local\Temp\ipykernel_18348\408301109.py:5: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Apart Deal.csv")


,지역코드,법정동,거래일,아파트,지번,전용면적,층,건축년도,거래금액
0,31110.0,학성동,5/30/2020 0:00,남운학성타운,379,135.5800,8,1991.0,26700
1,31110.0,남외동,1/3/2020 0:00,남외푸르지오1차,506-1,101.6000,2,2006.0,35500
2,31110.0,남외동,1/3/2020 0:00,에일린의뜰,500,84.9920,11,2007.0,36500
3,31110.0,남외동,1/3/2020 0:00,남외푸르지오1차,506-1,118.7060,8,2006.0,43000
4,31110.0,남외동,1/4/2020 0:00,남외푸르지오2차,501-1,84.9636,7,2007.0,38700


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002839 entries, 0 to 5002838
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   지역코드    float64
 1   법정동     object 
 2   거래일     object 
 3   아파트     object 
 4   지번      object 
 5   전용면적    float64
 6   층       object 
 7   건축년도    float64
 8   거래금액    object 
dtypes: float64(3), object(6)
memory usage: 343.5+ MB


In [58]:
df.describe()

,지역코드,전용면적,건축년도
count,5.002839e+06,5.002839e+06,5.002763e+06
mean,3.562552e+04,7.533272e+01,2.001677e+03
std,1.144216e+04,2.607859e+01,9.460423e+00
min,1.111000e+04,9.260000e+00,1.961000e+03
25%,2.820000e+04,5.971000e+01,1.995000e+03
50%,4.128100e+04,7.671810e+01,2.001000e+03
75%,4.374500e+04,8.496000e+01,2.009000e+03
max,5.013000e+04,4.243200e+02,2.023000e+03


In [59]:
df = df.drop(['법정동', '아파트', '지번'], axis=1)
df.head()

,지역코드,거래일,전용면적,층,건축년도,거래금액
0,31110.0,5/30/2020 0:00,135.5800,8,1991.0,26700
1,31110.0,1/3/2020 0:00,101.6000,2,2006.0,35500
2,31110.0,1/3/2020 0:00,84.9920,11,2007.0,36500
3,31110.0,1/3/2020 0:00,118.7060,8,2006.0,43000
4,31110.0,1/4/2020 0:00,84.9636,7,2007.0,38700


In [60]:
# 지역 코드 구분 
#areacode: 시/도
# gecode: 군/읍/면/동/구

areacode = [] #  반복문 전 리스트 초기화 
gecode = []


print(str(df['지역코드'][10])[0:2])

for code in df['지역코드']:
    acode = str(code)[0:2]
    gcode = str(code)[2:]
    areacode.append(int(acode))
    gecode.append(int(acode))

df['지역'] = gecode
df['지역코드'] = areacode

print(df[df['지역코드'] ==11][:10]) # 서울 11 경기 41 

31
        지역코드        거래일    전용면적   층    건축년도    거래금액  지역
655312    11  1/13/2018  140.78  15  2004.0  130000  11
655313    11  1/19/2018  103.77   5  2003.0   96000  11
655314    11  1/23/2018  123.13  10  2004.0  118000  11
655315    11  1/30/2018   66.96  13  1994.0   54500  11
655316    11  1/31/2018   76.66  11  1994.0   64000  11
655317    11  1/19/2018   29.76   3  1999.0   17900  11
655318    11  1/19/2018   29.76   4  1999.0   17900  11
655319    11  1/18/2018  104.73   3  1983.0   66000  11
655320    11  1/30/2018   59.34   3  1999.0   22000  11
655321    11  1/25/2018   84.90  10  1995.0   73000  11


In [61]:
#서울
df2 = df[df['지역코드'] ==11]
#경기
df3 = df[df['지역코드'] ==41]

df = pd.concat([df2,df3])
df.head()

,지역코드,거래일,전용면적,층,건축년도,거래금액,지역
655312,11,1/13/2018,140.78,15,2004.0,130000,11
655313,11,1/19/2018,103.77,5,2003.0,96000,11
655314,11,1/23/2018,123.13,10,2004.0,118000,11
655315,11,1/30/2018,66.96,13,1994.0,54500,11
655316,11,1/31/2018,76.66,11,1994.0,64000,11


In [62]:
# ‘거래일' 데이터 형식 통일
# ‘거래일’의 데이터 형식을 모두 ‘20150000’ 형식으로 통일하고, 후에 카테고리 데이터로 설정하여 예측에 용이한 형태로 변환될 수 있도록 구성
# 날짜 형식 변경
# '1/31/2018'을 '20180118'로 변경
import re


day = []


for date in df["거래일"]:
    date = date.split(" ")
    date = date[0]
    date = re.split("/|-", date)
    if len(date[0]) == 4:
        date = int(date[0] + date[1] + date[2])
    else:
        if len(date[0]) == 1:
            date[0] = "0" + date[0]
            if len(date[1]) == 1:
                date[1] = "0" + date[1]
        date = int(date[2] + date[0] + date[1])
    day.append(date)
df["거래일"] = day

df.head()

,지역코드,거래일,전용면적,층,건축년도,거래금액,지역
655312,11,20180113,140.78,15,2004.0,130000,11
655313,11,20180119,103.77,5,2003.0,96000,11
655314,11,20180123,123.13,10,2004.0,118000,11
655315,11,20180130,66.96,13,1994.0,54500,11
655316,11,20180131,76.66,11,1994.0,64000,11


In [63]:
# 거래 금액을 일반적인 숫자형 데이터로 변환
# 모두 문자형으로 변환한 후 replace( ) 함수로 ,(콤마)를 지우고 다시 숫자형으로 변환해 준다.
# 마지막으로 층의 Null 값을 0으로 채워 준다.
# 거래금액 콤마(,) 제거


df["거래금액"] = df["거래금액"].astype("str")
df["거래금액"] = df["거래금액"].str.replace(",", "")
df["거래금액"] = df["거래금액"].astype("int")


In [64]:
# 층 데이터가 비어있으면 0으로 채용 
df['층'] = pd.to_numeric(df['층'], errors='coerce').fillna(0)

In [65]:
# 데이터 타입 변환
# 범주형('category') 데이터는 데이터가 적은 고유값으로 구성되어 있을 경우 메모리 사용량을 크게 줄일 수 있습니다.
df['지역코드'] = df['지역코드'].astype('category')
df['지역'] = df['지역'].astype('category')
df['건축년도'] = df['건축년도'].astype('category')
df['거래일'] = df['거래일'].astype('category')
df['층'] = df['층'].astype('category')
df = df[['지역코드', '지역', '건축년도', '거래일', '층', '전용면적', '거래금액']]


df.sort_values(by=['지역'], axis=0)

,지역코드,지역,건축년도,거래일,층,전용면적,거래금액
655312,11,11,2004.0,20180113,15.0,140.7800,130000
2918509,11,11,2008.0,20160924,13.0,119.0950,128500
2918510,11,11,2011.0,20160924,19.0,105.8900,130000
2918511,11,11,1979.0,20160924,3.0,160.3300,144000
2918512,11,11,2005.0,20160926,24.0,59.6060,90000
...,...,...,...,...,...,...,...
1064863,41,41,2006.0,20181227,2.0,84.5862,20000
1064862,41,41,1998.0,20181227,13.0,59.8300,13000
1064861,41,41,2017.0,20181226,3.0,74.9123,29100
1064859,41,41,2017.0,20181220,23.0,84.6797,32200


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2107212 entries, 655312 to 4869948
Data columns (total 7 columns):
 #   Column  Dtype   
---  ------  -----   
 0   지역코드    category
 1   지역      category
 2   건축년도    category
 3   거래일     category
 4   층       category
 5   전용면적    float64 
 6   거래금액    int32   
dtypes: category(5), float64(1), int32(1)
memory usage: 52.3 MB


In [67]:
df.isnull().sum()

지역코드     0
지역       0
건축년도    18
거래일      0
층        0
전용면적     0
거래금액     0
dtype: int64

In [68]:
df['지역코드'].describe

<bound method NDFrame.describe of 655312     11
655313     11
655314     11
655315     11
655316     11
           ..
4869944    41
4869945    41
4869946    41
4869947    41
4869948    41
Name: 지역코드, Length: 2107212, dtype: category
Categories (2, int64): [11, 41]>

In [69]:
df['건축년도'] = pd.to_numeric(df['건축년도'], errors='coerce').fillna(0)

In [70]:
df.isnull().sum()

지역코드    0
지역      0
건축년도    0
거래일     0
층       0
전용면적    0
거래금액    0
dtype: int64

In [71]:
# 필요 모듈 라이브러리 불러오기


# 머신러닝 모델인 랜덤 포레스트(Random Forest)를 사이킷런(sklearn) 라이브러리에서 불러오기
# 데이터를 훈련 데이터와 테스트 데이터로 효과적으로 분리하기 위해 train_test_split 모듈을 불러오고,
# 모델의 학습 정도를 높이고 성능 평가를 위해 k-fold 모듈과
# cross_validation(교차 검증), metrics(평가 척도) 모듈을 불러오고 랜덤 seed도 설정


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import metrics


In [72]:
# 입력 0~6  타겟 7

# 훈련 데이터와 테스트 데이터 나누기
# 지역 코드, 지역, 건축년도, 거래일, 층, 전용 면적 속성이 입력 데이터 X에 저장
# 거래 금액 속성이 타겟 이므로 Y에 저장
# 실거래가 데이터를 모델이 학습할 수 있는 형태인 넘파이(Numpy) 배열로 변환
# 8:2로 분할하여 각각 훈련 데이터 세트, 테스트 데이터 세트로 나누기


dataset = df.values
X = dataset[:, 0:6]
Y = dataset[:, 6]


X = np.asarray(X)
Y = np.asarray(Y)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

In [73]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((1685769, 6), (421443, 6), (1685769,), (421443,))

In [74]:
# k-fold 및 랜덤 포레스트 모델 구성
# 학습의 정확도를 높이기 위해서 k-fold 모듈을 사용하여 훈련 데이터 세트를 다시 다섯 개로 나누어서 다섯 번의 학습을 진행
# 각각의 학습에서 다섯 개 중 네 개의 데이터 세트를 train set, 나머지 한 개의 데이터 세트를 validation set로 구분
# 각각의 데이터 세트를 모두 네 번씩 학습


kfold = KFold(n_splits=5, shuffle=True, random_state=42)
forest = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)


In [75]:
# k-fold와 랜덤 포레스트 모델 학습 - 오래걸림
# k-fold를 적용하여 학습을 진행하면서, 각 학습마다 평가 척도로 validation set의 RMSE
# (Root Mean Squared Error, 평균제곱근오차)와 R^2를 출력


for train, test in kfold.split(X_train, Y_train):
    print("TRAIN:", train, "TEST:", test)
    x_train, x_test = X_train[train], X_train[test]
    y_train, y_test = Y_train[train], Y_train[test]

    forest.fit(x_train, y_train)

    y_pred = forest.predict(x_test)

    print(
        "validation split rmse : {:.4f}".format(
            np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        )
    )
    print("validation split R2 : {: .4f}".format(metrics.r2_score(y_test, y_pred)))


TRAIN: [      0       1       3 ... 1685766 1685767 1685768] TEST: [      2      11      14 ... 1685754 1685756 1685762]
validation split rmse : 11987.5464
validation split R2 :  0.8930
TRAIN: [      0       1       2 ... 1685764 1685765 1685767] TEST: [      5       7      13 ... 1685760 1685766 1685768]
validation split rmse : 11884.7827
validation split R2 :  0.8937
TRAIN: [      2       4       5 ... 1685766 1685767 1685768] TEST: [      0       1       3 ... 1685757 1685758 1685764]
validation split rmse : 11966.0863
validation split R2 :  0.8934
TRAIN: [      0       1       2 ... 1685765 1685766 1685768] TEST: [      4       6       9 ... 1685759 1685761 1685767]
validation split rmse : 11740.7956
validation split R2 :  0.8962
TRAIN: [      0       1       2 ... 1685766 1685767 1685768] TEST: [      8      16      21 ... 1685750 1685763 1685765]
validation split rmse : 11877.4468
validation split R2 :  0.8945


## 예측(회귀) 평가 척도 ##


- 어떤 예측 인공지능 모델 결과 실제 값과 예측 값의 차이 즉, 오차가 각각 -2와 2가 발생했다고 가정
- 오차의 합은 0이지만 각각 오차가 존재한다.
- 이와 같이 오차는 부호가 있기 때문에 오차 평가를 위해 부로흘 처리해야 한다.
- 부호를 처리하는 방법은 다음과 같다
 - MAE(Mean Absolute Error): 평균절대오차 - 오차에 절대값을 씌어 평균을 낸 값으로, 예측 평가 척도 중 하나지만 절댓값을 사용해 미분할 수 없다는 점 때문에 활용이 어려워 잘 사용하지 않는다.
 - MSE(Mean Squared Error): 평균제곱오차 - 오차를 제곱한 후 평균을 낸 값
 - RSME(Root Mean Squared Error): 평균제곱근오차 - 예측 값과 실제 값 차이의 제곱에 대하여 평균을 낸 뒤 루트를 적용한 값으로 MSE가 급격히 커지는 왜곡을 줄여준다.
 - R^2: 결정계수 - 실제 값의 분산 대비 예측 값의 분산 비율로, 1에 가까울수록 좋은 모델, 0에 가까울수록 나쁜 모델로 평가된다. 음수가 나오면 잘못 평가된 값이다.

In [76]:
# $ cross_validate 모듈을 이용하여 학습 후에 훈련 데이터가 학습이 잘 되었는지, 시간은 얼마나 걸렸는지를 확인
cross_validate(
    forest, X_train, Y_train, cv=kfold, scoring=["r2"], return_train_score=True
)


{'fit_time': array([227.24457335, 232.33893347, 227.10402107, 223.88021159,
        225.24791408]),
 'score_time': array([45.76710272, 32.71097398, 35.68316293, 34.63364792, 42.31185937]),
 'test_r2': array([0.89296119, 0.89370768, 0.89342891, 0.89617446, 0.89445668]),
 'train_r2': array([0.98509282, 0.98498915, 0.98496662, 0.98506729, 0.98493163])}

In [77]:
# K-fold와 랜덤 포레스트 모델 학습 평가
# k-fold와 cross_validate로 랜덤 포레스트 모델의 학습과 평가를 마친 후, 테스트 데이터를 모델에 적용시켜 최종적으로 학습이 잘 되었는지 평가
# 마찬가지로 평가 척도는 RMSE와 R2를 사용하는데, 상대적인 평가가 가능한 평가 척도인 R2 결과가 0.96가 넘는 것으로 보아 학습이 잘 이루어졌음을 판단

print("테스트 세트 정확도: {:.3f}".format(forest.score(X_test, Y_test)))
pred = forest.predict(X_test)
print("rmse : {:.4f}".format(np.sqrt(metrics.mean_squared_error(Y_test, pred))))
print("R2 : {:.4f}".format(metrics.r2_score(Y_test, pred)))

# 정확도는 0.96이고, 오차 범위는 약 1억 853 만원 정도로 평가할 수 있다


테스트 세트 정확도: 0.894
rmse : 11853.0071
R2 : 0.8939


In [78]:
# 테스트 데이터로 가격 예측
# 학습을 거치지 않은 낯선 데이터는 테스트 데이터를 넣고 예측할 가격과 비교

for i in range(0, 20):
    print("실제 아파트 가격 : %d만원, 예측 아파트 가격 : %d만원" % (Y_test[i], pred[i]))


실제 아파트 가격 : 25000만원, 예측 아파트 가격 : 24708만원
실제 아파트 가격 : 29500만원, 예측 아파트 가격 : 30231만원
실제 아파트 가격 : 246000만원, 예측 아파트 가격 : 230349만원
실제 아파트 가격 : 26350만원, 예측 아파트 가격 : 25843만원
실제 아파트 가격 : 82300만원, 예측 아파트 가격 : 80265만원
실제 아파트 가격 : 24000만원, 예측 아파트 가격 : 24499만원
실제 아파트 가격 : 59000만원, 예측 아파트 가격 : 63542만원
실제 아파트 가격 : 160000만원, 예측 아파트 가격 : 167674만원
실제 아파트 가격 : 93000만원, 예측 아파트 가격 : 68023만원
실제 아파트 가격 : 19400만원, 예측 아파트 가격 : 24913만원
실제 아파트 가격 : 22000만원, 예측 아파트 가격 : 28948만원
실제 아파트 가격 : 45000만원, 예측 아파트 가격 : 99725만원
실제 아파트 가격 : 90000만원, 예측 아파트 가격 : 89465만원
실제 아파트 가격 : 21600만원, 예측 아파트 가격 : 21703만원
실제 아파트 가격 : 49000만원, 예측 아파트 가격 : 52383만원
실제 아파트 가격 : 28000만원, 예측 아파트 가격 : 28858만원
실제 아파트 가격 : 16600만원, 예측 아파트 가격 : 14803만원
실제 아파트 가격 : 36000만원, 예측 아파트 가격 : 28747만원
실제 아파트 가격 : 42000만원, 예측 아파트 가격 : 36254만원
실제 아파트 가격 : 29000만원, 예측 아파트 가격 : 30344만원


In [79]:
# XGBoost 모델 설정
# 의사결정 트리 기반의 Boost 모델인 XGBoost 알고리즘으로 학습
# XGBoost는 자체 오픈 소스 라이브러리로 불러오기

import xgboost
from xgboost import XGBRegressor


In [80]:
# k-fold를 적용하여 학습을 진행하고 XGBoost 모델을 구성
# XGBoost 모델은 랜덤 포레스트 알고리즘의 매개 변수와 동일하며 학습률(learning_rate)을 조절할 수 있는데, 여기서는 가장 일반적인 0.1로 설정

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
xgb = XGBRegressor(max_depth=20, learning_rate=0.1, n_estimators=100, n_jobs=-1)


In [81]:
# XGBoost 모델 설정
for train, test in kfold.split(X_train, Y_train):
    print("TRAIN:", train, "TEST:", test)
    x_train, x_test = X_train[train], X_train[test]
    y_train, y_test = Y_train[train], Y_train[test]

    xgb.fit(x_train, y_train)

    y_pred2 = xgb.predict(x_test)

    print(
        "validation split rmse : {:.4f}".format(
            np.sqrt(metrics.mean_squared_error(y_test, y_pred2))
        )
    )
    print("validation split R2 : {: .4f}".format(metrics.r2_score(y_test, y_pred2)))


TRAIN: [      0       1       3 ... 1685766 1685767 1685768] TEST: [      2      11      14 ... 1685754 1685756 1685762]
validation split rmse : 15124.4402
validation split R2 :  0.8296
TRAIN: [      0       1       2 ... 1685764 1685765 1685767] TEST: [      5       7      13 ... 1685760 1685766 1685768]
validation split rmse : 15338.9001
validation split R2 :  0.8229
TRAIN: [      2       4       5 ... 1685766 1685767 1685768] TEST: [      0       1       3 ... 1685757 1685758 1685764]
validation split rmse : 15184.8329
validation split R2 :  0.8284
TRAIN: [      0       1       2 ... 1685765 1685766 1685768] TEST: [      4       6       9 ... 1685759 1685761 1685767]
validation split rmse : 15072.9552
validation split R2 :  0.8289
TRAIN: [      0       1       2 ... 1685766 1685767 1685768] TEST: [      8      16      21 ... 1685750 1685763 1685765]
validation split rmse : 15199.0398
validation split R2 :  0.8272


In [82]:
print("테스트 세트 정확도: {:.3f}".format(xgb.score(X_test, Y_test)))
pred2 = xgb.predict(X_test)
print("rmse : {:.4f}".format(np.sqrt(metrics.mean_squared_error(Y_test, pred2))))
print("R2 : {:.4f}".format(metrics.r2_score(Y_test, pred2)))

# 결과를 보면 랜덤 포레스트가 더 좋음 

테스트 세트 정확도: 0.825
rmse : 15205.5438
R2 : 0.8254


In [83]:
for i in range(0, 20):
    print(
        "실제 아파트 가격 : %d만원, 예측 아파트 가격 : %d만원" % (Y_test[i], pred2[i])
    )

    # 평균 계산
    actual_avg = sum(Y_test) / len(Y_test)  # 실제 가격 평균
    predicted_avg = sum(pred2) / len(pred2)  # 예측 가격 평균

    # 변동 비율 계산
    change_rate = ((predicted_avg - actual_avg) / actual_avg) * 100

# 결과 출력
print(f"실제 아파트 가격 평균: {actual_avg:.2f}만원")
print(f"예측 아파트 가격 평균: {predicted_avg:.2f}만원")
print(f"변동 비율: {change_rate:.2f}%")


실제 아파트 가격 : 25000만원, 예측 아파트 가격 : 23968만원
실제 아파트 가격 : 29500만원, 예측 아파트 가격 : 30330만원
실제 아파트 가격 : 246000만원, 예측 아파트 가격 : 222021만원
실제 아파트 가격 : 26350만원, 예측 아파트 가격 : 28563만원
실제 아파트 가격 : 82300만원, 예측 아파트 가격 : 73986만원
실제 아파트 가격 : 24000만원, 예측 아파트 가격 : 23992만원
실제 아파트 가격 : 59000만원, 예측 아파트 가격 : 116350만원
실제 아파트 가격 : 160000만원, 예측 아파트 가격 : 156912만원
실제 아파트 가격 : 93000만원, 예측 아파트 가격 : 64874만원
실제 아파트 가격 : 19400만원, 예측 아파트 가격 : 25509만원
실제 아파트 가격 : 22000만원, 예측 아파트 가격 : 23361만원
실제 아파트 가격 : 45000만원, 예측 아파트 가격 : 164730만원
실제 아파트 가격 : 90000만원, 예측 아파트 가격 : 64889만원
실제 아파트 가격 : 21600만원, 예측 아파트 가격 : 20581만원
실제 아파트 가격 : 49000만원, 예측 아파트 가격 : 47756만원
실제 아파트 가격 : 28000만원, 예측 아파트 가격 : 20008만원
실제 아파트 가격 : 16600만원, 예측 아파트 가격 : 14015만원
실제 아파트 가격 : 36000만원, 예측 아파트 가격 : 27239만원
실제 아파트 가격 : 42000만원, 예측 아파트 가격 : 29797만원
실제 아파트 가격 : 29000만원, 예측 아파트 가격 : 28877만원
실제 아파트 가격 평균: 44303.51만원
예측 아파트 가격 평균: 44295.09만원
변동 비율: -0.02%
